In [1]:
import dataclasses
import requests
import logging
import base64
import pandas as pd
import openai
import os
from dotenv import load_dotenv

import nature_go_client
from question_generation import generate_questions

logger = logging.getLogger(__name__)

BASE_URL = 'http://nature-go.edouardleurent.com'
LOGIN_URL = BASE_URL + '/api/auth/login/'
SPECIES_LIST_URL = BASE_URL + '/api/species/all/'
SPECIES_DETAIL_URL = BASE_URL + '/api/species/{species_id}/'
QUESTION_CREATE_URL = BASE_URL + '/api/university/quiz/questions/'

# Your id here
NG_USERNAME = 'nathangrinsztajn'
NG_PASSWORD = 'vesper'

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
client = nature_go_client.NatureGoClient(username=NG_USERNAME, password=NG_PASSWORD)
client.login()

## Retrieve species with missing questions through Nature go API

In [3]:
species_list = client.get_all_species()
questions_list = client.get_all_questions()

id_with_questions = set([q['species'] for q in questions_list])
species_without_questions = [s for s in species_list if not s['id'] in id_with_questions]

print(f'Found {len(species_without_questions)} species without illustrations out of {len(species_list)} total species')
pd.DataFrame(species_without_questions)

Found 1 species without illustrations out of 2 total species


,id,commonNames,scientificName,scientificNameWithoutAuthor,genus,family,descriptions,illustration,illustration_transparent,wikipedia_image_url,illustration_url,display_name
0,2,"[Yellow Mountain Saxifrage, Evergreen Saxifrag...",Saxifraga aizoides L.,Saxifraga aizoides,Saxifraga,Saxifragaceae,[],http://nature-go.edouardleurent.com/media/spec...,None,,http://nature-go.edouardleurent.com/media/spec...,Yellow Mountain Saxifrage


## Run generation with openai API

In [4]:
species_name, species_id = species_without_questions[0]['scientificName'], species_without_questions[0]['id']
questions = generate_questions(species_name)

In [5]:
questions

[{'question': 'How does Saxifraga aizoides spread and grow?',
  'choices': ['Through tall stems',
   'By forming large trees',
   'Via short rhizomes, forming mats of small colonies',
   'By producing long vines'],
  'correct_choice': 2},
 {'question': 'What color are the flowers of Saxifraga aizoides?',
  'choices': ['Red', 'Blue', 'Yellow-green', 'Purple'],
  'correct_choice': 2},
 {'question': 'Where is Saxifraga aizoides found?',
  'choices': ['Only in North America',
   'Only in Europe',
   'In both North America and Europe',
   'Only in New York state'],
  'correct_choice': 2},
 {'question': 'What is the conservation status of Saxifraga aizoides in New York state?',
  'choices': ['Endangered', 'Threatened', 'Extinct', 'Least Concern'],
  'correct_choice': 1}]

## Upload result to Nature Go API

In [6]:
client.post_species_questions(species_id, questions)

ValueError: http://nature-go.edouardleurent.com/api/university/quiz/questions/ failed with status  201 and response b'{"id":8,"species":2,"question":"What is the conservation status of Saxifraga aizoides in New York state?","choices":["Endangered","Threatened","Extinct","Least Concern"],"correct_choice":1}'

In [ ]:
# 201 is actually success for post